## EchoTube: Your Personal GEN-AI for YouTube Summaries & Q&A

In [7]:
import os
import sys
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

custom_package_path = '/content/drive/MyDrive/my_colab_packages'

if custom_package_path not in sys.path:
    sys.path.insert(0, custom_package_path)

print(f"Custom package path added to sys.path: {custom_package_path}")
print(f"Current sys.path: {sys.path}")

Custom package path added to sys.path: /content/drive/MyDrive/my_colab_packages
Current sys.path: ['/content/drive/MyDrive/my_colab_packages', '/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpmmdfeje6']


In [3]:
!pip install -q youtube-transcript-api langchain-community langchain-text-splitters faiss-cpu tiktoken python-dotenv pytube

import os
import re
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, RequestBlocked
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from google.colab import drive
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [8]:


# Define the path where models are stored
model_storage_path = '/content/drive/MyDrive/colab_models'

# --- Helper Functions ---
def get_youtube_video_id(url):
    """
    Extracts the YouTube video ID from a given YouTube URL.
    """
    if "youtube.com/watch?v=" in url:
        match = re.search(r"v=([\w-]+)", url)
    elif "youtu.be/" in url:
        match = re.search(r"youtu.be/([\w-]+)", url)
    elif "youtube.com/embed/" in url:
        match = re.search(r"embed/([\w-]+)", url)
    elif "youtube.com/v/" in url:
        match = re.search(r"v/([\w-]+)", url)
    else:
        return None

    if match:
        return match.group(1)
    return None

def load_embedding_model_interactive():
    embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
    local_embedding_model_dir = os.path.join(model_storage_path, embedding_model_id.split('/')[-1])

    if not os.path.exists(local_embedding_model_dir):
        print(f"Error: Embedding model not found at {local_embedding_model_dir}. Please ensure it's downloaded and available.")
        return None

    print(f"Loading embedding model: {embedding_model_id}...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=local_embedding_model_dir,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': False}
    )
    print("Embedding model loaded!")
    return embedding_model

def load_llm_model_interactive():
    llm_model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
    local_llm_model_dir = os.path.join(model_storage_path, llm_model_id.split('/')[-1])

    if not os.path.exists(local_llm_model_dir):
        print(f"Error: LLM model not found at {local_llm_model_dir}. Please ensure it's downloaded and available.")
        return None

    print(f"Loading LLM model: {llm_model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(local_llm_model_dir)
    model = AutoModelForCausalLM.from_pretrained(local_llm_model_dir)
    # pipe = pipeline(
    #     "text-generation",
    #     model=model,
    #     tokenizer=tokenizer,
    #     max_new_tokens=150,
    #     temperature=0.7,
    #     top_p=0.9,
    #     pad_token_id=tokenizer.eos_token_id
    # )
    # llm = HuggingFacePipeline(pipeline=pipe)
    # Create a Hugging Face pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        temperature=0.5,
        max_new_tokens=500, # Set a reasonable limit for generated text
        # Other pipeline arguments can be added here as needed
    )

    # Wrap the pipeline with HuggingFacePipeline for LangChain compatibility
    llm = HuggingFacePipeline(pipeline=pipe)

    model = ChatHuggingFace(llm=llm)


    #print("LLM model loaded!")
    return model

# --- Load models once at the beginning ---
print("\n--- Initializing Models ---")
embedding_model = load_embedding_model_interactive()
llm = load_llm_model_interactive()

if embedding_model is None or llm is None:
    print("Exiting due to model loading errors. Please ensure models are correctly downloaded.")
else:
    print("\n--- YouTube Video Q&A ---")
    youtube_url = input("Enter YouTube Video URL (or 'exit' to quit): ")




    #user_question = input("Ask a question about the video: ")

    if not youtube_url:
        print("Error: Please enter a YouTube video URL.")

    else:
        print("\nProcessing video and generating answer...")
        video_id = get_youtube_video_id(youtube_url)

        if not video_id:
            print("Error: Could not extract video ID from the provided URL. Please check the URL.")
        else:
            transcript_text = ""
            try:
                # Clear any existing proxy environment variables to avoid conflicts
                if 'http_proxy' in os.environ: del os.environ['http_proxy']
                if 'https_proxy' in os.environ: del os.environ['https_proxy']

                ytt_api = YouTubeTranscriptApi()
                transcript_text = " ".join([snippet.text for snippet in ytt_api.fetch(video_id)])
                if not transcript_text:
                    print("Warning: Transcript fetched but it's empty. Cannot answer questions.")
            except TranscriptsDisabled:
                print("Error: Transcripts are disabled for this video.")
            except NoTranscriptFound:
                print("Error: No transcript found for this video in available languages.")
            except RequestBlocked:
                print("Error: YouTube blocked the transcript request. This often happens from cloud servers. Consider using a proxy if this persists.")
            except Exception as e:
                print(f"Error: An unexpected error occurred while fetching transcript: {e}")

            if transcript_text:
                # --- Indexing (Chunking and Vector Store) ---
                splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
                chunks = splitter.create_documents([transcript_text])

                vector_store = FAISS.from_documents(chunks, embedding_model)

                # --- Retrieval ---
                retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})


                # --- Augmentation ---
                # context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
                # prompt = PromptTemplate(
                #     template="""
                #       You are a helpful assistant.
                #       Answer ONLY from the provided transcript context. If the context is insufficient, state that you don't have enough information.

                #       Context: {context}
                #       Question: {question}
                #     """,
                #     input_variables=['context', 'question']
                # )
                # while True:
                #   user_question = input("Ask a question about the video: ")
                #   if not user_question:
                #     print("Error: Please enter a question.")
                #   if user_question.lower() == 'exit':
                #     print("Exiting Q&A session.")
                #     break
                #   final_prompt_value = prompt.invoke({"context": context_text, "question": user_question})
                #   final_prompt = final_prompt_value.text

                #   # --- Generation ---
                #   try:
                #       answer = llm.invoke(final_prompt)

                #       # Clean up potentially repeated prompt from TinyLlama output
                #       cleaned_answer = answer.content.split('Question: ' + user_question)[-1].strip()
                #       cleaned_answer = cleaned_answer.replace('\n', ' ')
                #       # ANSI escape code for white text: \033[97m, and reset: \033[0m
                #       print(f"\n--- Answer ---\n\033[97m{cleaned_answer}\033[0m")
                #   except Exception as e:
                #       print(f"Error generating answer from LLM: {e}")
                #   else:
                #     print("Cannot answer questions without a transcript.")


--- Initializing Models ---
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2...
Embedding model loaded!
Loading LLM model: TinyLlama/TinyLlama-1.1B-Chat-v1.0...


Device set to use cuda:0



--- YouTube Video Q&A ---
Enter YouTube Video URL (or 'exit' to quit): https://youtu.be/WSDcYUrGODE?si=ducQBdEgt0HwJK4G

Processing video and generating answer...


In [9]:
transcript_text = " ".join([snippet.text for snippet in ytt_api.fetch(video_id)])
print(transcript_text)

What are Newton's three laws of motion? Sir Isaac Newton was a famous scientist that lived almost 400 years ago. He is famous for learning and understanding how and why objects move. Like when an apple falls from a tree. Because he figured out why objects move. We call them Newton's three laws of motion. The first is called the Law of inertia and explains why things move. It says that an object at rest stays at rest, and an object in motion stays in motion at its same speed and direction, unless acted upon by another force. So this ball just sitting in the grass won't start moving on its own. It'll just sit there at rest. But if I kick it, my foot would be applying a force to it. And the ball would move. Newton's second law talks about how objects change when a force acts on it. This law says the acceleration of an object is directly proportional to the net force acting on it, and inversely proportional to its mass. Sounds pretty confusing, doesn't it? Indeed. Which brings us to Newton

In [10]:
while True:

  user_question = input("\n Ask a question about the video: ")
  if not user_question:
    print("Error: Please enter a question.")
  if user_question.lower() == 'exit':
    print("Exiting Q&A session.")
    break
  retrieved_docs = retriever.invoke(user_question)
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  prompt = PromptTemplate(
                    template="""
                      You are a helpful assistant.
                      Answer ONLY from the provided transcript context. If the context is insufficient, state that you don't have enough information.

                      Context: {context}
                      Question: {question}
                    """,
                    input_variables=['context', 'question']
                    )
  final_prompt_value = prompt.invoke({"context": context_text, "question": user_question})
  final_prompt = final_prompt_value.text

  # --- Generation ---
  try:
      answer = llm.invoke(final_prompt)

      # Clean up potentially repeated prompt from TinyLlama output
      cleaned_answer = answer.content.split('Question: ' + user_question)[-1].strip()
      cleaned_answer = cleaned_answer.replace('\n', ' ')
      # ANSI escape code for white text: \033[97m, and reset: \033[0m
      print(f"--- Answer ---:{cleaned_answer}")
  except Exception as e:
      print(f"Error generating answer from LLM: {e}")
  else:
    print("Cannot answer questions without a transcript.")


 Ask a question about the video: what is newton first law?
--- Answer ---:</s> <|assistant|> Newton's first law of motion is called the Law of Inertia and explains why objects move in a straight line unless a force is applied to them. The first law states that an object at rest remains at rest, and an object in motion stays in motion at the same speed and direction unless acted upon by a force.
Cannot answer questions without a transcript.

 Ask a question about the video: explain me newton second law
--- Answer ---:</s> <|assistant|> The second law of motion, as explained by Sir Isaac Newton, is a fundamental principle of physics that states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. This law is also known as the law of universal gravitation and explains why objects move in a predictable and consistent way. The action and reaction of a force are always equal in strength, but the direction of the fo